# create_temp_reports.

In [112]:
import sys
sys.path.append('../../')
import glob
import shutil
from pathlib import Path
import os

import numpy as np
import pandas as pd
from config import CLEAN_DATA_DIR, OUTPUT_DIR, RAW_DATA_DIR
import db as db
from scripts.normalize_taxa import add_normalized_name_column

from scripts.normalize_data import (
    check_duplicate_columns
)

In [43]:
clean_data_path = CLEAN_DATA_DIR

metadata_file = OUTPUT_DIR/'metadata'/'LIMS'/'Micropal_changes.csv' 
# metadata_file = OUTPUT_DIR/'metadata'/'LIMS'/'Micropal_changes_4.csv' 


date = '2022-04-28'


taxa_list_file = OUTPUT_DIR/'taxa'/'LIMS'/f"taxa_list_{date}.csv"
taxa_crosswalk_list_file = OUTPUT_DIR/'taxa'/'LIMS'/f"taxa_crosswalk_{date}.csv"

PI_4_file = OUTPUT_DIR /'taxa'/'LIMS'/f'PI_Micropal_CSV_4_normalized_taxa_list_with_pbdb_{date}.csv'
additional_taxa_path = OUTPUT_DIR/'taxa'/'LIMS'/'addtional_species.csv'
PI_file = RAW_DATA_DIR/'PI_processed_files'/f'LIMS_Micropal_headers_PBDB_Taxonomy_notes_taxa_list_{date}.csv'


## create a file with all columns  in LIMS 4

In [4]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_group,change_file_encoding,remove_empty_rows,remove_spaces_from_columns,delete_duplicate_colums_with_spaces,delete_duplicate_rows,delete_renamed_duplicate_columns,remove_bad_characters
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,False,False,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,False,False,False,False,False,False,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,False,False,False,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,True,False,False,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,False,False,False,False


In [14]:
data = []
for path in metadata['path']:
    if '317_U1351_planktic_forams.csv' in str(path):
        header = 1
    else:
        header = 0
    df = pd.read_csv(clean_data_path/path, dtype=str, header=header)
    df.dropna(axis="columns", how='all', inplace=True)
    for index, row in df.iterrows():
        row_data = {"path": path}
        for col in df.columns:
            row_data[col] = row[col]
        data.append(row_data)

In [15]:
len(data)

6818

In [16]:
new_df = pd.DataFrame(data)
new_df.shape

(6818, 1712)

In [17]:
new_df = new_df[sorted(new_df.columns)]


In [18]:
new_df.to_csv('../../output/tmp/combine_mircopal_4.csv')

# LIMS stats for paper

In [139]:
sql = """
 select count(*) as count 
 from taxa ;
"""

row = db.fetch_one(sql)
print(  row['count'])

4675


In [84]:
sql = """
 select count(*) as count, taxon_group 
 from taxa
 group by taxon_group;
"""

rows = db.fetch_all(sql)
for row in rows:
    print(row['taxon_group'],  row['count'])

benthic_forams 1510
bolboformids 2
chrysophyte_cysts 1
diatoms 654
dinoflagellates 53
ebridians 6
nannofossils 825
ostracods 15
other 11
palynology 133
planktic_forams 844
radiolarians 598
silicoflagellates 23


In [151]:
sql = """
 select count(*) 
 from taxa
where taxon_name_above_genus is not null;
"""

row = db.fetch_one(sql)
print( row['count'])

108


In [155]:
sql = """
 select count(distinct(genus_name || taxon_group))
 from taxa
where genus_name is not null
;
"""
row = db.fetch_one(sql)
print( row['count'])

1080


In [157]:
sql = """
 select count(distinct(name || taxon_group))
 from taxa
where species_name is not null
 ;
"""
row = db.fetch_one(sql)
print( row['count'])

4567


In [158]:
sql = """
 select count(distinct(name || taxon_group))
 from taxa
where subspecies_name is not null
 ;
"""
row = db.fetch_one(sql)
print( row['count'])

138


In [95]:
lims_df = pd.read_csv(PI_file, header=9)
lims_df = lims_df.drop(0)
lims_df.dropna(axis=0, how='all', inplace=True)
add_normalized_name_column(lims_df)

lims_df.head(2)

,taxon_group,verbatim_name,name,"name to use (if different from ""name"")",name comment field,Comment,Notes (change to Internal only notes?),Any taxon above genus,genus modifier,genus name,...,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,normalized_name
1,benthic_forams,Pyrite,Pyrite,NaN,NaN,"not a taxa; Going into Macrostrat, not PBDB",Summer 2020: deal with later; should go in as ...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,benthic_forams,Euuvigerina miozea (group) >100 m,Euuvigerina miozea (group) >100 m,Euuvigerina miozea,NaN,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",NaN,NaN,Euuvigerina,...,NaN,NaN,NaN,NaN,NaN,288974.0,Foraminifera,212476.0,Rhizaria,Euuvigerina miozea


In [96]:
lims_4_df = pd.read_csv(PI_4_file)
lims_4_df.dropna(axis=0, how='all', inplace=True)
add_normalized_name_column(lims_4_df)

lims_4_df.head(2)

,taxon_group,verbatim_name,Name,Comment,Notes (change to Internal only notes?),Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,...,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,abbrev_genus,normalized_name
0,benthic_forams,Agglutinated Foram. gen. et sp. indet.,Textulariia indet.,NaN,NaN,Textulariia indet.,NaN,NaN,NaN,NaN,...,NaN,NaN,288974.0,Foraminifera,212476.0,Rhizaria,NaN,NaN,False,Textulariia indet.
1,benthic_forams,Agglutinated indet.,Textulariia indet.,NaN,NaN,Textulariia indet.,NaN,NaN,NaN,NaN,...,NaN,NaN,288974.0,Foraminifera,212476.0,Rhizaria,NaN,NaN,False,Textulariia indet.


In [97]:
add_df = pd.read_csv(additional_taxa_path)
add_df.dropna(axis=0, how='all', inplace=True)

add_df.head(2)

,sample_id,file,sample,taxon_group,verbatim_name,code,Name,Comment,Notes (change to Internal only notes?),Any taxon above genus,...,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,normalized_name,path
0,203528,374_U1525A_radiolarians.csv,374-U1525A-1H-1-IW_MUDLINE,radiolarians,additional species: Lonchosphaera spicata,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lonchosphaera spicata,LIMS/Micropal_CSV_2/374_U1525A_radiolarians.csv
1,203528,374_U1525A_radiolarians.csv,374-U1525A-1H-1-IW_MUDLINE,radiolarians,additional species: Poulpus spp.-like,NaN,Poulpus spp.,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Poulpus spp.,LIMS/Micropal_CSV_2/374_U1525A_radiolarians.csv


In [132]:
cols = ['verbatim_name', 'taxon_group', 'normalized_name']

lims_df_2 = lims_df[cols]
lims_4_df_2 = lims_4_df[cols]
add_df_2 = add_df[cols]

combine = pd.concat([lims_df_2, lims_4_df_2, add_df_2])
combine.loc[combine['normalized_name'] == '', 'normalized_name'] = np.nan

combine.shape

(5453, 3)

In [133]:
combine[combine['normalized_name'].isna()].shape

(23, 3)

In [134]:

combine2 = combine[['normalized_name', 'taxon_group']]
combine2 = combine2.dropna(subset=['normalized_name'])

combine2.drop_duplicates().shape

(4675, 2)